In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import matplotlib.pyplot as plt
import datetime

In [2]:
# Load cellids into lists

astro_list = [33487935587, 42658809302, 51189618418]
microglia_list = [60142133362]
neuro_list = [6745384646, 5931028416]
segment_list = [42265647259, 42396836391, 42411044227, 42426053615, 42484370274, 42629401999, 42731915687, 51262199575, 51276888038, 59837235207, 68934592617, 68934651026, 68949223223, 77770855109, 77770869291, 77785427101, 86548800877, 86563402479, 86563416023, 86577944070, 86577958826, 86577988239, 86578017614, 86592530364, 86592530373, 86592530504, 86592530542, 86592530562, 86592530605, 86592530606, 86592530612, 86592530618, 86592530622, 86592530676, 86592530686, 86592530689, 86592530703, 86592530732, 86592530741, 86592530839, 86592530906, 86592545146, 86592545252, 86592545393, 86592545440, 86592545469, 86592545472, 86607073155, 86607087642, 86607087887, 86607102459, 86607102465]

print(f'number of astros:', len(astro_list))
print(f'number of astros:', len(microglia_list))
print(f'number of neurons:', len(neuro_list))
print(f'number of segments:', len(segment_list))

number of astros: 3
number of astros: 1
number of neurons: 2
number of segments: 52


In [3]:
mesh_dir = 'data/c3_cell_meshes/dec/' # or change to your desired folder
seg_source = "precomputed://gs://h01-release/data/20210601/c3"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [4]:
# Combine all the lists into a dictionary with their respective prefixes
prefix_dict = {
    'astro_': astro_list,
    'microglia_': microglia_list,
    'neuro_': neuro_list,
    'segment_': segment_list
}

# Make a dictionary to hold the mesh file for each cell id in the lists
mesh_dictionary = {}
for prefix, cell_list in prefix_dict.items():
    for cellid in cell_list:
        mesh_file = os.path.join(mesh_dir, str(cellid) + '_0.ply')
        
        # Error handling
        try:
            mesh_dictionary[prefix + str(cellid)] = trimesh.load_mesh(mesh_file)
        except FileNotFoundError:
            print(f"File not found: {mesh_file}, skipping...")
        except Exception as e:
            print(f"Error loading {mesh_file}: {e}, skipping...")


In [5]:
len(mesh_dictionary)

58

In [6]:
mesh_dictionary

{'astro_33487935587': <trimesh.Trimesh(vertices.shape=(1267026, 3), faces.shape=(2536342, 3), name=`33487935587_0.ply`)>,
 'astro_42658809302': <trimesh.Trimesh(vertices.shape=(983391, 3), faces.shape=(1954111, 3), name=`42658809302_0.ply`)>,
 'astro_51189618418': <trimesh.Trimesh(vertices.shape=(931432, 3), faces.shape=(1866332, 3), name=`51189618418_0.ply`)>,
 'microglia_60142133362': <trimesh.Trimesh(vertices.shape=(27627, 3), faces.shape=(55614, 3), name=`60142133362_0.ply`)>,
 'neuro_6745384646': <trimesh.Trimesh(vertices.shape=(293462, 3), faces.shape=(585571, 3), name=`6745384646_0.ply`)>,
 'neuro_5931028416': <trimesh.Trimesh(vertices.shape=(323703, 3), faces.shape=(643578, 3), name=`5931028416_0.ply`)>,
 'segment_42265647259': <trimesh.Trimesh(vertices.shape=(2472, 3), faces.shape=(4932, 3), name=`42265647259_0.ply`)>,
 'segment_42396836391': <trimesh.Trimesh(vertices.shape=(1465, 3), faces.shape=(2918, 3), name=`42396836391_0.ply`)>,
 'segment_42411044227': <trimesh.Trimesh(v

In [7]:
# Define opacities for different cell types
opacities = {
    'astro_': 0.01,
    'microglia_': 0.25,
    'neuro_': 0.25,
    'segment_': 0.5
}

# Create mesh actors with opacity and random colors based on cell type prefix
mesh_actor = {}
for cell_key, mesh in mesh_dictionary.items():
    # Determine the cell type based on the prefix
    prefix = None
    for key_prefix in opacities.keys():
        if cell_key.startswith(key_prefix):
            prefix = key_prefix
            break

    if prefix:
        random_color = list(np.random.random(size=3))  # Random color
        cell_opac = opacities[prefix]  # Opacity based on the prefix
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=random_color)

locals().update(mesh_actor)

# Create a camera object and define the view based on the first available mesh
if len(mesh_actor) > 0:
    first_cell_key = next(iter(mesh_actor))
    camera = trimesh_vtk.oriented_camera(mesh_dictionary[first_cell_key].centroid, backoff=800)
else:
    print("No valid mesh actors found in mesh_dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000026068322AD0) at 0x0000026060400708>

Save image to disk

In [8]:
save_dir = 'vtk_images/'
base_name = 'axon_whorl'
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=10,
#                           camera=camera
#                          )

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000026068321190) at 0x0000026005BDB7C8>